In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import preprocessing

## Model

In [2]:
def join(series):
    #return ' '.join( str(list(series)).strip('[]').split(', ') )
    return ' '.join( map(str, series) )

In [3]:
%run 'metrics.ipynb'
def model_evaluation(actual, prediction):
    return mapk(actual['ad_id'], prediction['ad_id'], k=12)

In [4]:
pd.read_csv("../generated/final/events_clicks_CTR_addocs_dist_no_w_train.csv", nrows=10).head()

,display_id,document_id,timestamp,traffic_source,platform,day,hour,geo,ad_id,clicked,adsPerDisplay,clicksPerShows,addoc_id,dist
0,8,1330329,638,1.0,2.0,1,7,2765,95724,0,4,0.034785,1136820,0.537786
1,8,1330329,638,1.0,2.0,1,7,2765,175694,0,4,0.202327,1392479,0.430210
2,8,1330329,638,1.0,2.0,1,7,2765,280430,1,4,0.256853,1601512,0.245120
3,8,1330329,638,1.0,2.0,1,7,2765,329774,0,4,0.052426,1652347,0.259652
4,2657,1271490,185255,1.0,2.0,1,7,2765,70081,0,4,0.150145,933594,0.124739


In [5]:
usecols = ['display_id', 'ad_id', 'clicksPerShows', 'dist', 'clicked']
#usecols = ['display_id', 'ad_id', 'clicksPerShows', 'platform', 'traffic_source', 'dist', 'clicked']
features = ['clicksPerShows', 'dist']
#features = ['clicksPerShows', 'clicksPerShowsM']
#features = ['p1', 'p2', 'p3', 't1', 't2', 't3', 'adsPerDisplay']

In [6]:
pd.read_csv("../generated/final/events_clicks_CTR_addocs_dist_no_w_train.csv", nrows=10, usecols=usecols).head()

,display_id,ad_id,clicked,clicksPerShows,dist
0,8,95724,0,0.034785,0.537786
1,8,175694,0,0.202327,0.430210
2,8,280430,1,0.256853,0.245120
3,8,329774,0,0.052426,0.259652
4,2657,70081,0,0.150145,0.124739


## Preparation

### Step 1

In [7]:
train_nrows, test_nrows = 650000, 210000 #  None, None #

In [8]:
train_index = pd.read_csv("../generated/final/train_index.csv", nrows=train_nrows)['index'].ravel()
test_index = pd.read_csv("../generated/final/test_index.csv", nrows=test_nrows)['index'].ravel()

In [9]:
nrows = max(train_index[-1], test_index[-1])+1
nrows

866741

In [10]:
train = pd.read_csv("../generated/final/events_clicks_CTR_addocs_dist_no_w_train.csv", usecols = usecols, nrows=nrows)
train.count()

display_id        866741
ad_id             866741
clicked           866741
clicksPerShows    866741
dist              790180
dtype: int64

In [11]:
dist_mean = train['dist'].mean()
train = train.fillna({'dist':dist_mean}, inplace=True)

In [12]:
test = train.ix[test_index].copy()
train = train.ix[train_index].copy()
train.count()

display_id        650000
ad_id             650000
clicked           650000
clicksPerShows    650000
dist              650000
dtype: int64

In [13]:
test.count()

display_id        210000
ad_id             210000
clicked           210000
clicksPerShows    210000
dist              210000
dtype: int64

In [14]:
trainM = pd.DataFrame({'clicksPerShows_min':train.groupby('display_id')['clicksPerShows'].min(),
                       'clicksPerShows_max':train.groupby('display_id')['clicksPerShows'].max(),
                       'dist_min':train.groupby('display_id')['dist'].min(),
                       'dist_max':train.groupby('display_id')['dist'].max()
                      }).reset_index()
trainM.count()

display_id            122656
clicksPerShows_max    122656
clicksPerShows_min    122656
dist_max              122656
dist_min              122656
dtype: int64

In [15]:
trainM['clicksPerShows_l'] = trainM['clicksPerShows_max'] - trainM['clicksPerShows_min']
trainM['dist_l'] = trainM['dist_max'] - trainM['dist_min']
trainM.drop(['clicksPerShows_max', 'dist_max'], axis=1, inplace=True)
#trainM.drop(['clicksPerShows_max', 'dist_min'], axis=1, inplace=True)
trainM.count()

display_id            122656
clicksPerShows_min    122656
dist_min              122656
clicksPerShows_l      122656
dist_l                122656
dtype: int64

In [16]:
print sum(trainM['clicksPerShows_l'] == 0)
print sum(trainM['dist_l'] == 0)

1
10219


In [17]:
trainM.loc[trainM['clicksPerShows_l'] == 0, 'clicksPerShows_l'] = 1
trainM.loc[trainM['dist_l'] == 0, 'dist_l'] = 1

In [18]:
train = train.merge(trainM, on='display_id')
train.count()

display_id            650000
ad_id                 650000
clicked               650000
clicksPerShows        650000
dist                  650000
clicksPerShows_min    650000
dist_min              650000
clicksPerShows_l      650000
dist_l                650000
dtype: int64

In [19]:
del trainM

In [20]:
train['clicksPerShows'] = ( train['clicksPerShows'] - train['clicksPerShows_min'] ) / train['clicksPerShows_l']
train['dist'] = ( train['dist'] - train['dist_min'] ) / train['dist_l']
#train['dist'] = ( train['dist_max'] - train['dist'] ) / train['dist_l']
train.drop(['clicksPerShows_min', 'clicksPerShows_l', 'dist_min', 'dist_l'], axis=1, inplace=True)
#train.drop(['clicksPerShows_min', 'clicksPerShows_l', 'dist_max', 'dist_l'], axis=1, inplace=True)
train.count()

display_id        650000
ad_id             650000
clicked           650000
clicksPerShows    650000
dist              650000
dtype: int64

In [21]:
testM = pd.DataFrame({'clicksPerShows_min':test.groupby('display_id')['clicksPerShows'].min(),
                       'clicksPerShows_max':test.groupby('display_id')['clicksPerShows'].max(),
                       'dist_min':test.groupby('display_id')['dist'].min(),
                       'dist_max':test.groupby('display_id')['dist'].max()
                      }).reset_index()
testM.count()

display_id            39656
clicksPerShows_max    39656
clicksPerShows_min    39656
dist_max              39656
dist_min              39656
dtype: int64

In [22]:
testM['clicksPerShows_l'] = testM['clicksPerShows_max'] - testM['clicksPerShows_min']
testM['dist_l'] = testM['dist_max'] - testM['dist_min']
testM.drop(['clicksPerShows_max', 'dist_max'], axis=1, inplace=True)
#testM.drop(['clicksPerShows_max', 'dist_min'], axis=1, inplace=True)
testM.count()

display_id            39656
clicksPerShows_min    39656
dist_min              39656
clicksPerShows_l      39656
dist_l                39656
dtype: int64

In [23]:
print sum(testM['clicksPerShows_l'] == 0)
print sum(testM['dist_l'] == 0)

0
3235


In [24]:
testM.loc[testM['clicksPerShows_l'] == 0, 'clicksPerShows_l'] = 1
testM.loc[testM['dist_l'] == 0, 'dist_l'] = 1

In [25]:
test = test.merge(testM, on='display_id')
del testM
test.count()

display_id            210000
ad_id                 210000
clicked               210000
clicksPerShows        210000
dist                  210000
clicksPerShows_min    210000
dist_min              210000
clicksPerShows_l      210000
dist_l                210000
dtype: int64

In [26]:
test['clicksPerShows'] = ( test['clicksPerShows'] - test['clicksPerShows_min'] ) / test['clicksPerShows_l']
test['dist'] = ( test['dist'] - test['dist_min'] ) / test['dist_l']
#test['dist'] = ( test['dist_max'] - test['dist'] ) / test['dist_l']
test.drop(['clicksPerShows_min', 'clicksPerShows_l', 'dist_min', 'dist_l'], axis=1, inplace=True)
#test.drop(['clicksPerShows_min', 'clicksPerShows_l', 'dist_max', 'dist_l'], axis=1, inplace=True)
test.count()

display_id        210000
ad_id             210000
clicked           210000
clicksPerShows    210000
dist              210000
dtype: int64

In [27]:
scaler = preprocessing.StandardScaler()
excpt = [] #['clicksPerShows', 'distM'] #, 'p1', 'p2', 'p3', 't1', 't2', 't3', 'adsPerDisplay']
for f in features:
    if not f in excpt:
        print f
        scaler.fit(train[[f]])
        train[f] = scaler.transform(train[[f]]) 
        test[f] = scaler.transform(test[[f]])

clicksPerShows
dist


In [28]:
train[features].head()

,clicksPerShows,dist
0,-1.106690,1.507749
1,0.876369,0.571487
2,1.521752,-1.039403
3,-0.897889,-0.912922
4,-0.029751,-1.039403


## Validation

In [29]:
clf = linear_model.SGDClassifier(random_state=0, loss='log', fit_intercept=False, n_iter=5)
clf.fit(train[features], train['clicked'].ravel())

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=False, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=0, shuffle=True, verbose=0,
       warm_start=False)

In [30]:
clf.classes_

array([0, 1], dtype=int64)

In [31]:
clicks_test_sample = test[['display_id', 'ad_id', 'clicked']]
clicks_test_sample.count()

display_id    210000
ad_id         210000
clicked       210000
dtype: int64

In [32]:
validation = clicks_test_sample[clicks_test_sample['clicked']==1].astype({'ad_id':str}).sort_values('display_id')
validation.count()

display_id    39655
ad_id         39655
clicked       39655
dtype: int64

In [33]:
test[features].head()

,clicksPerShows,dist
0,1.521752,-1.039403
1,-1.106690,1.166379
2,-0.988214,0.631064
3,0.755502,0.058999
4,-0.939184,1.507749


In [34]:
test['res'] = pd.Series(clf.predict_proba(test[features])[:,1])
test.head()

,display_id,ad_id,clicked,clicksPerShows,dist,res
0,19972,120162,1,1.521752,-1.039403,0.713018
1,19972,122279,0,-1.106690,1.166379,0.339528
2,19972,168342,0,-0.988214,0.631064,0.356491
3,19972,176585,0,0.755502,0.058999,0.609895
4,19972,250080,0,-0.939184,1.507749,0.361433


In [35]:
clf.coef_, clf.intercept_[0]

(array([[ 0.59216395, -0.00862235]]), 0.0)

In [36]:
prediction = test.sort_values(by=['display_id', 'res'], ascending=[True, False])
prediction.count()

display_id        210000
ad_id             210000
clicked           210000
clicksPerShows    210000
dist              210000
res               210000
dtype: int64

In [37]:
prediction=pd.DataFrame({ 'ad_id': prediction.groupby('display_id')['ad_id'].apply(join) }).astype({'ad_id':str}).reset_index()
prediction.count()

display_id    39656
ad_id         39656
dtype: int64

In [38]:
prediction.head()

,display_id,ad_id
0,120,153192 116987
1,197,78098 128496 300898 187797 65119 70943
2,796,143302 158632 202812 145293 285991 235440
3,1132,155232 149541 234445 67305 85397 150083 328479...
4,1681,258183 98740 86836 145293


In [39]:
validation.head()

,display_id,ad_id,clicked
85967,120,116987,1
85971,197,78098,1
76930,796,202812,1
85978,1132,150083,1
83577,1681,258183,1


In [40]:
print 'clicksPerShows :', model_evaluation(validation, prediction)  #0.662869594202      0.588499968978      

clicksPerShows : 0.588785527156


In [41]:
#['clicksPerShows']                             0.662869594202       0.588499968978 
#['clicksPerShows', 'dist']                     0.66282443126        0.588638094838     -> 0.662839173432 (niter=20)
                                                                    #0.588785527156
                                      #         0.6628185045   +platform
                                     #          0.662806944913 +intercept
#['clicksPerShows', 'dist', 'adsPerDisplay']                         0.588542728654                                     
#['clicksPerShows', 'p1', 'p2', 'p3', 'dist']                        0.588165686987
#['clicksPerShows', 'dist', 't1', 't2', 't3']                        0.588181057654
#['clicksPerShows', 'p1', 'p2', 'p3']                                0.588499968978
#['clicksPerShows', 't1', 't2', 't3']                                0.588499968978 
#['clicksPerShows', 'adsPerDisplay']                                 0.588499968978


In [42]:
1+1

2